# Resumama
Resumama leverages a fine-tuned T5-small transformer model to analyze job postings and generate one-page, tailored resumes optimized for each application. By predicting the fit between a candidate's long-format resume and a given job post, Resumama provides a three-tiered recommendation: "Good Fit," "Potential Fit," or "No Fit." For suitable matches, the system then generates a concise, professional resume highlighting the most relevant qualifications and experiences.

In [1]:
import torch

print("CUDA Available:", torch.cuda.is_available())
print("CUDA Version:", torch.version.cuda)
print("Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")


CUDA Available: True
CUDA Version: 11.8
Device Name: NVIDIA GeForce RTX 2060


### T5-Small
T5-small is a compact version of the Text-to-Text Transfer Transformer (T5) model, developed by Google. It treats every NLP task as a text-to-text problem, enabling it to perform tasks such as translation, summarization, and classification within a unified framework. With 60 million parameters, T5-small balances computational efficiency and performance, making it suitable for resource-constrained applications while maintaining robust language understanding capabilities.

In [ ]:
# model = model.to("cpu")
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to("cuda")



You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Data Preparation

The data used for training contains pairs of resume texts and job descriptions, annotated with labels indicating the fit level ("Good Fit," "Potential Fit," or "No Fit"). It includes a total of 4,992 samples, with the majority labeled as "No Fit" (3,143), followed by "Potential Fit" (1,556), and "Good Fit" (1,542). Each record consists of a candidate's resume, a corresponding job description, and a fit label, allowing the model to learn the relationship between resumes and job requirements. The class imbalance in the dataset, with more "No Fit" examples, may present challenges for training, potentially requiring rebalancing techniques or weighting adjustments to improve classification performance.

In [4]:
import pandas as pd

splits = {'train': 'train.csv', 'test': 'test.csv'}
df = pd.read_csv("hf://datasets/cnamuangtoun/resume-job-description-fit/" + splits["train"])

In [5]:
df['label'].value_counts()

No Fit           3143
Potential Fit    1556
Good Fit         1542
Name: label, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
'''
preprocess_data takes in a dataframe and a tokenizer and returns a dictionary of inputs and outputs that are ready to be passed to the model
'''
# Prepare the data
def preprocess_data(df, tokenizer, max_length=512):
    inputs = []
    labels = []
    
    for _, row in df.iterrows():
        input_text = (f"Evaluate the fit level of the following resume for the job description. Understand what makes a resume a good fit."
                      f"Job Description: {row['job_description_text']} "
                      f"Resume: {row['resume_text']}")
        label_text = row['label']
        
        inputs.append(input_text)
        labels.append(label_text)

    # Tokenize the inputs and outputs
    tokenized_inputs = tokenizer(inputs, max_length=max_length, padding=True, truncation=True, return_tensors="pt")
    tokenized_labels = tokenizer(labels, max_length=max_length, padding=True, truncation=True, return_tensors="pt")

    # Return tokenized inputs and labels
    return tokenized_inputs, tokenized_labels

# Split data into train and validation
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Tokenize
tokenizer = T5Tokenizer.from_pretrained("t5-small")
train_inputs, train_labels = preprocess_data(train_df, tokenizer)
val_inputs, val_labels = preprocess_data(val_df, tokenizer)


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
train_df


,resume_text,job_description_text,label
3659,SummaryA business management graduate with sig...,Position Title: Senior Accountant Organization...,Potential Fit
1426,Professional ProfileCapable International Tax ...,RoleGaming Business AnalystResponsibilities Ab...,No Fit
497,Professional ProfileHighly motivated Sales Ass...,If you can handle the accounting responsibilit...,No Fit
2833,SummaryOrganized and motivated employee eager ...,Our client is a growing Medical Device company...,No Fit
1480,SummaryEmployed by the U.S. Navy as a Civilian...,Were seeking a detail-oriented and analytical ...,No Fit
...,...,...,...
3772,Professional ProfileExpert in Functional Testi...,Lead Software Developer New York City - Hybr...,Potential Fit
5191,SummaryI am a detail-oriented professional see...,Job Purpose: Perform designated tasks in the a...,Good Fit
5226,SummaryDedicated and focused Clerk who excels ...,Role - Business Analyst - Mobile Location - Lo...,Good Fit
5390,SummaryTo obtain a challenging and rewarding a...,"About Allvue\nWe are Allvue Systems, the leadi...",Good Fit


In [ ]:
import torch
''' 
fit_level_dataset is a custom dataset class that takes in tokenized inputs and labels and returns a dictionary of input_ids, attention_mask, and labels
'''
class FitLevelDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings.input_ids[idx],
            "attention_mask": self.encodings.attention_mask[idx],
            "labels": self.labels.input_ids[idx],
        }

train_dataset = FitLevelDataset(train_inputs, train_labels)
val_dataset = FitLevelDataset(val_inputs, val_labels)


### Fine-Tune
Here we fine-tune a pre-trained T5-small model on our resume dataset. The T5ForConditionalGeneration class is used to load the pre-trained model, which is capable of generating text conditioned on input text. Training arguments are configured via TrainingArguments, specifying parameters like the output directory, evaluation strategy, batch size, number of epochs, learning rate, and precision mode. The Trainer class orchestrates the training process, combining the model, training arguments, and datasets (training and validation). Finally, the trainer.train() call initiates the fine-tuning process, updating the model weights based on the input data to optimize its performance.

In [9]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

# Load the T5 model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Training arguments
training_args = TrainingArguments(
    output_dir="./t5-fit-level",
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    logging_dir="./logs",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=10,
    learning_rate=3e-5,
    save_total_limit=2,
    fp16=True,  # Enable mixed precision if you have GPU
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()


c:\Users\Michael\Anaconda3\envs\tf-gpu\lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/49920 [00:00<?, ?it/s]

{'loss': 1.79, 'grad_norm': 2.7435436248779297, 'learning_rate': 2.9703725961538463e-05, 'epoch': 0.1}


  0%|          | 0/1249 [00:00<?, ?it/s]

{'eval_loss': 0.6606224179267883, 'eval_runtime': 22.0997, 'eval_samples_per_second': 56.517, 'eval_steps_per_second': 56.517, 'epoch': 0.1}
{'loss': 0.5865, 'grad_norm': 7.690234661102295, 'learning_rate': 2.9403245192307694e-05, 'epoch': 0.2}


...

  0%|          | 0/1249 [00:00<?, ?it/s]

{'eval_loss': 0.7406260967254639, 'eval_runtime': 21.909, 'eval_samples_per_second': 57.009, 'eval_steps_per_second': 57.009, 'epoch': 9.82}
{'loss': 0.4384, 'grad_norm': 4.763878345489502, 'learning_rate': 2.692307692307692e-07, 'epoch': 9.92}


  0%|          | 0/1249 [00:00<?, ?it/s]

{'eval_loss': 0.7377873063087463, 'eval_runtime': 22.1981, 'eval_samples_per_second': 56.266, 'eval_steps_per_second': 56.266, 'epoch': 9.92}
{'train_runtime': 6253.5418, 'train_samples_per_second': 7.983, 'train_steps_per_second': 7.983, 'train_loss': 0.45593070066892183, 'epoch': 10.0}


TrainOutput(global_step=49920, training_loss=0.45593070066892183, metrics={'train_runtime': 6253.5418, 'train_samples_per_second': 7.983, 'train_steps_per_second': 7.983, 'total_flos': 6756262729482240.0, 'train_loss': 0.45593070066892183, 'epoch': 10.0})

#### Training result
The fine-tuned T5-small model achieved a training loss of 0.456, indicating that it effectively learned patterns from the training data while minimizing prediction errors. Over the course of 10 epochs, the model completed 49,920 steps, processing approximately 8 samples per second, which demonstrates efficient utilization of computational resources. The reported floating-point operations (FLOPs) totaled approximately 6.76 quadrillion, highlighting the computational intensity of the training process.

In [10]:
def predict(job_description, resume):
    input_text = (f"Evaluate the fit level of the following resume for the job description: "
                  f"Job Description: {job_description} "
                  f"Resume: {resume}")
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids,top_p=0.9)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return prediction

# Example prediction
job_description = "Software Engineer with experience in Python and cloud computing."
resume = "Experienced developer with expertise in Python, AWS, and software design."
print(predict(job_description, resume))


Good Fit


c:\Users\Michael\Anaconda3\envs\tf-gpu\lib\site-packages\transformers\generation\configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
c:\Users\Michael\Anaconda3\envs\tf-gpu\lib\site-packages\transformers\generation\utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [10]:
torch.cuda.empty_cache()

### Job description Parsing

Here we will use Optical Character Recognition with the PaddleOCR library to extract text from an image. It initializes the PaddleOCR object. The ocr method processes the given image path and returns a structured result, from which the function extracts only the recognized text (ignoring bounding box data and confidence scores). The extracted text is concatenated into a single string and returned, providing a readable transcription of the text present in the image. This is particularly useful for converting job description images into machine-readable text for further analysis.

In [17]:
import pytesseract

# Set tesseract_cmd to the Tesseract executable path
pytesseract.pytesseract.tesseract_cmd = r"c:\users\michael\anaconda3\envs\tf-gpu\lib\site-packages"


In [ ]:
from paddleocr import PaddleOCR

def extract_text_with_paddleocr(image_path):
    ocr = PaddleOCR()  # Initialize PaddleOCR
    results = ocr.ocr(image_path)
    text = [line[1][0] for line in results[0]]  # Extract the text content
    return " ".join(text)

# Example Usage
job_description = extract_text_with_paddleocr("images/job_description_image.jpg")
print(job_description)


[2024/12/07 14:44:06] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\Michael/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\Michael/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=N

Here we extracts all text from a given PDF file using the PyPDF2 library. It initializes a PdfReader object to read the PDF, iterates through each page, and appends the extracted text to a string variable. Finally, the function returns the concatenated text, providing a plain-text representation of the entire resume.

In [12]:
from PyPDF2 import PdfReader
# Extract original resume text from a PDF
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    resume_text = ""
    for page in reader.pages:
        resume_text += page.extract_text()
    return resume_text


### Generate Resume
 Here we atempt to create a tailored resume. We provide an input prompt combining the job description and the original resume, instructing the model to generate a tailored version. The prompt is tokenized into a format suitable for the model and transferred to a GPU for faster processing. Using the model's generate method, it produces a single output sequence, ensuring the response stays within a specified maximum length of 512 tokens. Finally, the generated text is decoded into a readable format and returned as the tailored resume, optimized for the provided job description.

In [17]:
# Generate a tailored resume based on the job description and original resume
def generate_matching_resume(job_description, resume_text):
    # Construct the input prompt for the model
    input_text = (
        f"Generate a tailored Resume\n\n"
        f"Job Description: {job_description}\n\n"
        f"Original Resume: {resume_text}"
    )
    input_ids = tokenizer(input_text, return_tensors="pt", truncation=True).input_ids.to("cuda")
    
    # Generate the tailored resume
    outputs = model.generate(input_ids, max_length=512, num_return_sequences=1)
    tailored_resume = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return tailored_resume

In [ ]:
job_description_image = "images/job_description_image.jpg"  # Replace with the actual job description image file path
resume_pdf_path = "resources/Resume.pdf"  # Replace with the actual resume PDF file path

# Step 1: Extract text from the job description image
job_description = extract_text_with_paddleocr(job_description_image)
print("Extracted Job Description:")
print(job_description)

# Step 2: Extract text from the resume PDF
original_resume_text = extract_text_from_pdf(resume_pdf_path)
print("\nExtracted Original Resume Text:")
print(original_resume_text)

# Step 3: Generate a tailored resume
tailored_resume = generate_matching_resume(job_description, original_resume_text)
print("\nTailored Resume:")
print(tailored_resume)

[2024/12/07 14:45:44] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\Michael/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\Michael/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=N

### Conclusion
The model's underperformance in generating tailored resumes could stem from several factors. First, fine-tuning the T5-small model may have been insufficient for generating coherent, professional resumes due to the complexity of resume formatting and content prioritization. The training data may not have provided enough diverse examples of high-quality resumes and their alignment with job descriptions, limiting the model's ability to generalize effectively. Additionally, truncation of input data during tokenization could have caused the model to lose critical context from longer resumes or job descriptions

Another contributing factor could be the lack of explicit training for formatting resumes in a professional structure, as T5 is primarily optimized for text-to-text tasks rather than structured document generation. The model also likely struggled to balance creativity with precision, essential for accurately reflecting a candidate's qualifications while adhering to job requirements.

Future efforts could include expanding the dataset with more annotated examples of resumes and job descriptions, ensuring a diverse representation of industries and roles. Incorporating additional pre-processing steps, such as segmenting resumes into discrete sections, could provide the model with more structured input. Fine-tuning larger transformer models, like T5-base or T5-large, might improve performance by leveraging their greater capacity to understand and generate complex content.